<a href="https://colab.research.google.com/github/zeeshansalim1234/Summer2021/blob/main/Pachira_nan_reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers
!pip install tweepy
!pip install bert-extractive-summarizer
!pip install nltk
!pip install google-cloud-vision

     |████████████████████████████████| 92kB 11.4MB/s 
     |████████████████████████████████| 2.5MB 34.3MB/s 
     |████████████████████████████████| 1.2MB 49.7MB/s 
     |████████████████████████████████| 3.3MB 39.6MB/s 
     |████████████████████████████████| 901kB 45.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=9d9ab2b59955f470150fe49fab33eb8fbd4baf1a457b29a82bb9c46f2c639542
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.
  Found existing installation: huggingface-hub 0.0.13
    Uninstalling huggingface-hub-0.0.13:
      Successfully uninstalled huggingface-hub-0.0.13
     |████████████████████████████████| 368kB 27.5MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 


In [2]:
import pandas as pd
import json
import os
from sentence_transformers import SentenceTransformer, util
from google.colab import drive
import tweepy
import re
import os,io
from google.cloud import vision
from google.cloud.vision_v1 import types
import pandas as pd
import nltk

ACCESS_TOKEN="1401579178072231936-98mY1wOw1UR3GHOjdo4ePnfdfUHt6n"
ACESS_TOKEN_SECRET="QiIslicgx8GNGrGDPrx8xNZc2WoCyrUsZumgXSyGzGLOx"
CONSUMER_KEY="uaiStuv7EYdYxWSomPKHvSSF5"
CONSUMER_SECRET="JOQryy37w9HPMSrSw8msyyb048iqeHmK4xCRyWP1oBhLKwLYlb"

auth=tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACESS_TOKEN_SECRET)
api=tweepy.API(auth)

#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'seismic-diorama-316110-5569927e0d86.json'
#client = vision.ImageAnnotatorClient()

In [3]:
def pre_process(tweets):

    for i in range(0, len(tweets)):

        if (tweets[i] is not None):

            if(tweets[i]!=tweets[i]):
                tweets[i]=""

            tweets[i] = tweets[i].lower()  # To lower case
            tweets[i] = tweets[i].replace('@','')  # remove @
            tweets[i] = tweets[i].replace('#','')  # remove #
            tweets[i] = remove_urls(tweets[i])  # remove URL
            tweets[i] = remove_emojis(tweets[i])  # remove emojis
            tweets[i] = "".join(j for j in tweets[i] if j not in (
            "?", ".", ";", ":", "!", "-", ",", "[", "]", "(", ")", "’", "‘", '"', "$", "'", "“", "”", "•", "=", "+",
            "%", "/", "&", "|", "~"))  # remove punctuations
            tweets[i] = removeNonEnglishWordsFunct(tweets[i])

    return tweets

def remove_urls (str):

    str = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', str, flags=re.MULTILINE)
    return(str)


def remove_emojis(data):

    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoji, '', data)

def removeNonEnglishWordsFunct(x):

    new_string=re.sub('[^a-zA-Z0-9]',' ',x)

    cleaned_string=re.sub('\s+',' ',new_string)
    return cleaned_string

In [17]:
import numpy as np
df = pd.read_csv('/content/charities_ab.csv')

#charities =df.CharityName.values+" "+df.OngoingPrograms.values

df = df.dropna()

charities=df.CharityName.values+" "+df.OngoingPrograms.values
cities=df.City.values

print(len(charities), "Charities")

#We then load the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')

#To encode the descriptions to a single string
charity_texts = [charity for charity in charities]
cities_texts=[city for city in cities]

#Compute embeddings for all descriptions
corpus_embeddings = model.encode(charity_texts, convert_to_tensor=True)


1562 Charities


In [22]:
print(charity_texts[0])

"POLANIE" POLISH SONG & DANCE ASSOCIATION Performance for public: Concerts, Festivals, Private InvitesTeaching Polish folk songs and dance to people with polish roots and/or interest in polish cultureBall fundraising, casino fundraising, bake sale fundraising, mother day fundraising


In [23]:

# Returns all charities at the provided location

def search_papers_location(location):

   results=[]
   for i in range(0,len(cities_texts)):
     if(cities_texts[i]==location):
       results.append(i)
   return results

  

In [37]:
# Generates 100 random charities as the priority

import random
from termcolor import colored

#Generate 5 random numbers between 10 and 30
priority = random.sample(range(0, 1520), 100)
print(priority)


[677, 767, 552, 1406, 454, 37, 1440, 386, 486, 199, 1005, 95, 837, 316, 1462, 927, 643, 303, 27, 1280, 400, 54, 371, 443, 379, 679, 526, 147, 1055, 1263, 1108, 553, 1377, 634, 193, 556, 897, 521, 993, 134, 728, 23, 764, 745, 1387, 805, 321, 929, 42, 931, 1328, 423, 623, 684, 320, 1030, 122, 892, 133, 871, 1158, 8, 505, 294, 775, 1252, 1235, 1184, 315, 329, 1456, 390, 401, 178, 186, 879, 449, 441, 761, 641, 427, 826, 103, 461, 1492, 1471, 166, 380, 958, 1197, 1434, 473, 388, 757, 1476, 1321, 383, 359, 1267, 735]


In [30]:
# For generating recommendations for priority organizations

def generate_priority(title):

   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings,top_k=10)
   search_hits = search_hits[0]
   
   count = 0
   top_header = "\n\nPriority Charities\n\n"
   print(colored(top_header,'yellow'))

   flag=0

   for hit in search_hits:

          for i in range(0,len(priority)):
            if(hit['corpus_id']==priority[i]):
              flag=1
              break

          if(flag==1 and search_hits[count]['score']>0.70):
            related_charities = charities[hit['corpus_id']]
            count += 1
            subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms']== related_charities]
            k=subsetDataFrame.values

            print("\n"+str(count)+") "+colored(str(k[0][0]), 'red'))
            print("similiarity score of " + str(format(search_hits[count-1]['score'],".2f")))
            print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
            flag=0
            count+=1


In [31]:

# For Generating recommendation for all organizations

def search_papers(title,city_indices,status,num_results):

   #status signifies city 0 is city based
   #num_results signifies the number of results we wish to generate

   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings,top_k=10)
   search_hits = search_hits[0]
   
   count = 0
   flag=0
   tracker=0 #basically counter for the non location segment



   for hit in search_hits:
        
        if(status==0) :

          for i in range(0,len(city_indices)):
            if(hit['corpus_id']==city_indices[i]):
              flag=1
              break

          if(flag==1):

            related_charities = charities[hit['corpus_id']]
            count += 1
            subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms']== related_charities]
            k=subsetDataFrame.values
            

            print("\n"+str(count)+") "+colored(str(k[0][0]), 'red'))
            print("similiarity score of " + str(format(search_hits[count-1]['score'],".2f")))
            print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
            flag=0

        else:

          if(tracker<num_results):
            related_charities = charities[hit['corpus_id']]
            count += 1
            subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms'] == related_charities]
            k=subsetDataFrame.values
            
          
            print("\n"+str(count)+") "+colored(str(k[0][0]), 'red'))
            print("similiarity score of " + str(format(search_hits[count-1]['score'],".2f")))
            print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
            flag=0
            tracker+=1


#want to make a big impact donations made through birdies4kids for the alberta diabetes foundation are matched up to 50  birdies for kids runs until august 15th 2021  to donate and read more 

#alberta wins reddeer sisters donate land to nature conservancy of canada ncc preserve a haven for species wildlife ruth dorothy bower donate 193 hectares of land on the west bank of the reddeer river bowerwildlifesanctuary

#in the news the kidney foundation encourages other canadian organizations and companies to to adopt policies to support living organdonors read more about kidneycanada s wage replacement policy for living organ and tissue donation

#participate in the scotiabank calgary marathon charity challenge to help children amp families at childrens cottage society theres an event for everyone of all abilities register or donate at  thanks for your consideration children families 

#can you help us many children need their very own books strong reading role models amp safe fun reading spaces your donation helps more children experience the magic of reading amp a lifetime of opportunity 

In [14]:
pip install pytrends


In [50]:
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US') #specify language

"""
data=pytrends.trending_searches('canada')
print(data[0].loc[0])
"""

pytrends.build_payload(kw_list=['protest'],geo='CA-AB',timeframe='now 7-d')  # generate the query
related_queries = pytrends.related_queries()
result=list(related_queries.values())
query=list(result[0].values())[1]
print(query)

#search_papers(query,[],1,5)

#list(result[0].values())[1].loc[0].at['query']

                          query  value
0              winnipeg protest   1550
1      tiananmen square protest    600
2    no farmers no food protest    600
3            canada day protest    190
4          calgary mask protest    150
5  edmonton legislature protest    130
6                    gwen berry    110
7     anti mask protest calgary     90
8           residential schools     60
9         farmers protest india     60


In [52]:
print("What basis do you want to generate a recommendation on ? ")
print("\n1. Enter a sentence  ")
print("2. Trending tweet on a particular hashtag ")
print("3. Image-based recommendation generation \n")
choice=input("Enter your choice : ")

if(choice=="1"):

  title=input("\nEnter a sentence : ")
  enable_loc=str(input("Do you wish to enable location ? "))

  pytrends = TrendReq(hl='en-US') #specify language
  pytrends.build_payload(kw_list=['protest'],geo='CA-AB',timeframe="now 7-d")  # generate the query
  related_queries = pytrends.related_queries()
  result=list(related_queries.values())
  query=list(result[0].values())[1].loc[2].at['query']
  print("\n"+query)
  
  top_header = "\n\nPopular in your Region\n\n"
  print(colored(top_header,'magenta'))
  search_papers(query,[],1,5)
  

  if(enable_loc=="yes"):

    status=0
    location=str(input("Enter location : "))
    loc_indices=search_papers_location(location)
    generate_priority(title)
    top_header = "\n\nTop 10 related charities\n\n"
    print(colored(top_header,'blue'))
    search_papers(title,loc_indices,status,10)

  else:

    pytrends = TrendReq(hl='en-US') #specify language
    pytrends.build_payload(kw_list=['protest'],geo='CA-AB')  # generate the query
    related_queries = pytrends.related_queries()
    result=list(related_queries.values())
    
    temp=[]
    status=1
    generate_priority(title)
    top_header = "\n\nTop 10 related charities\n\n"
    print(colored(top_header,'blue'))
    search_papers(title,temp,status,10)

elif(choice=="2"):

  tag = input("\nEnter a hashtag : ")
  tweets=tweepy.Cursor(api.search,q=tag,result_type='popular',tweet_mode="extended").items(1)
  temp=[]
  for tweet in tweets :

      temp.append(tweet.full_text)

  temp=pre_process(temp)
  tweet=temp[0]
  print("\nMost famous tweet : \n"+colored(tweet,'orange'))
  search_papers(tweet)

elif(choice=="3"):

  with io.open("horse.jpg", 'rb') as image_file:
    content = image_file.read()


  sample_tweet = """Beautiful Rose (R) and beautiful Sunflower (L) are seen here enjoying a breezy afternoon here at the ranch. 

  Sparkleshttp://linktr.ee/PRRHR

  #miniturehorses #horse #rescue #minihorsesoftwitter #BestFriendsDay #donate #tuesdayvibe"""

  sample_tweet=pre_process([sample_tweet])

  image = vision.Image(content=content)

  response_label = client.label_detection(image=image)
  response_text = client.text_detection(image=image)

  temp=""
  count=0

  for label in response_label.label_annotations:
      if(count<3):
        temp+=label.description+" "
        count+=1

  for r in response_text.text_annotations:
      temp+=r.description+" "


  temp=pre_process([temp])
  print("\n\nInterpretation of tweet : "+colored(sample_tweet[0]+" "+temp[0],'red'))
  search_papers(sample_tweet[0]+" "+temp[0])



#alberta wins reddeer sisters donate land to nature conservancy of canada ncc preserve a haven for species wildlife ruth dorothy bower donate 193 hectares of land on the west bank of the reddeer river bowerwildlifesanctuary


What basis do you want to generate a recommendation on ? 

1. Enter a sentence  
2. Trending tweet on a particular hashtag 
3. Image-based recommendation generation 

Enter your choice : 1

Enter a sentence : alberta wins reddeer sisters donate land to nature conservancy of canada ncc preserve a haven for species wildlife ruth dorothy bower donate 193 hectares of land on the west bank of the reddeer river bowerwildlifesanctuary
Do you wish to enable location ? no

no farmers no food protest


Popular in your Region



1) Cold Lake Food Bank Society
similiarity score of 0.84
Description of charity : Cold Lake Food Bank Society Food GivingMilk ProgramChristmas Hamper Project COLD LAKE

2) SOCIETY OF ST VINCENT DE PAUL ST BONVENTURE CONFERENCE CALGA
similiarity score of 0.81
Description of charity : SOCIETY OF ST VINCENT DE PAUL ST BONVENTURE CONFERENCE CALGA WE RUN A FOOD BANK TO ASSIST NEEDY FAMILIES IN OUR COMMUNITY. WE ALSO HELP WITH RENT PAYMENTS AND UTILITY BILLS, MEDICAL PRESCRIPTI

In [ ]:
"""
from newsapi import NewsApiClient

newsapi = NewsApiClient(api_key='90833a9050904d1c9cb011ddd785d357')

# /v2/top-headlines
top_headlines = newsapi.get_everything(q='global protest',sort_by='relevancy')
print(top_headlines['articles'][5]['title'])
search_papers(top_headlines['articles'][5]['title'],[],1)
"""
